In [1]:
import os
import ssl
import pg8000
import re

In [2]:
# conexion a RDS usando el bundle de Amazon, esto no es necesario desde lambda
RDS_CA_PATH = os.getenv(
    "RDS_SSL_CA",
    r"C:\Users\Julian Waksmann\Documents\03 UNIVERSIDAD\AlimentAPP\global-bundle.pem",
)

ssl_context = ssl.create_default_context(cafile=RDS_CA_PATH)

In [3]:
# Variables
ENV = 'dev'
PASSWORD_INPUT = input('ingrese contraseña')

In [4]:
def get_connection():
    """
    Abre una conexion pg8000 con SSL.
    """
    return pg8000.connect(
        host='db-alimentapp.cm7oyoiiqchb.us-east-1.rds.amazonaws.com',
        port=5432,
        database='postgres',
        user='labo_team',
        password=PASSWORD_INPUT,
        ssl_context=ssl_context,
        timeout=10,
    )

In [27]:
def run_query(sql: str):
    """
    Ejecuta un SELECT y devuelve filas como diccionarios.
    """
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    if not rows:
        return []
    columns = [desc[0] for desc in cur.description]
    return [dict(zip(columns, row)) for row in rows]

def run_command(sql: str):
    """
    Ejecuta INSERT/UPDATE/DELETE y hace commit.
    """
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

# Post nuevo pedido / orden de venta

In [76]:
import json
from typing import Any, Dict, List
from datetime import date

class ValidationError(Exception):
    """
    Error de validación para payloads de orden de venta.
    """

def create_production_orders(order_id: int, items: List[Dict[str, int]]) -> None:
    """
    Genera ordenes_produccion 'planificadas' para los productos recibidos.
    """
    for item in items:
        run_command(
            f"""
            INSERT INTO {ENV}.orden_produccion
                (id_orden_venta, id_producto, cantidad, estado)
            VALUES
                ({order_id}, {item['id_producto']}, {item['cantidad']}, 'planificada')
            """
        )
        
def validate_sale_order_payload(payload: Dict[str, Any]):
    """
    Valida que la orden tenga todos los campos obligatorios y datos correctos.
    """
    required = ["id_vendedor", "id_cliente", "productos", "fecha_entrega_solicitada"]
    missing = [key for key in required if not payload.get(key)]
    if missing:
        raise ValidationError(f"Faltan campos obligatorios: {', '.join(missing)}")

    fecha_entrega = payload["fecha_entrega_solicitada"]

    try:
        fecha_entrega_date = date.fromisoformat(fecha_entrega)
    except ValueError:
        raise ValidationError("fecha_entrega_solicitada debe tener formato AAAA-MM-DD.")

    if fecha_entrega_date <= date.today():
        raise ValidationError("fecha_entrega_solicitada debe ser posterior a la fecha actual.")
    
    if not run_query(f"SELECT 1 FROM {ENV}.cliente WHERE id = {int(payload['id_cliente'])}"):
        raise ValidationError("El id_cliente indicado no existe.")

    if not run_query(
        f"SELECT 1 FROM {ENV}.empleado WHERE id = {int(payload['id_vendedor'])} AND activo = TRUE"
    ):
        raise ValidationError("El id_vendedor indicado no existe o está inactivo.")

    productos = payload["productos"]
    if not isinstance(productos, list) or not productos:
        raise ValidationError("productos debe ser una lista con al menos un item.")

    items: List[Dict[str, int]] = []
    total = 0.0
    for item in productos:
        if "id_producto" not in item or "cantidad" not in item:
            raise ValidationError("Cada producto debe incluir id_producto y cantidad.")
        if item["cantidad"] <= 0:
            raise ValidationError("La cantidad de cada producto debe ser mayor a cero.")

        producto = run_query(
            f"SELECT precio_venta FROM {ENV}.producto "
            f"WHERE id = {int(item['id_producto'])} AND activo = TRUE"
        )
        if not producto:
            raise ValidationError(f"El producto {item['id_producto']} no existe o está inactivo.")

        precio_unit = float(producto[0]["precio_venta"])
        total += precio_unit * item["cantidad"]
        items.append(
            {"id_producto": int(item["id_producto"]), "cantidad": int(item["cantidad"])}
        )

    return {
        "id_cliente": int(payload["id_cliente"]),
        "id_vendedor": int(payload["id_vendedor"]),
        "fecha_entrega_solicitada": fecha_entrega,
        "productos": items,
        "total": total,
    }


In [77]:
def insert_sale_order(validated: Dict[str, Any]):
    """
    Inserta la orden de venta (estado 'pendiente') y devuelve el registro creado.
    """
    run_command(
        f"""
        INSERT INTO {ENV}.orden_venta
            (id_cliente, id_empleado, fecha_entrega_solicitada, estado, valor_total_pedido)
        VALUES
            ({validated['id_cliente']},
             {validated['id_vendedor']},
             '{validated['fecha_entrega_solicitada']}',
             'pendiente',
             {validated['total']})
        """
    )

    rows = run_query(
        f"""
        SELECT id, valor_total_pedido, estado, fecha_pedido
        FROM {ENV}.orden_venta
        WHERE id_cliente = {validated['id_cliente']}
          AND id_empleado = {validated['id_vendedor']}
        ORDER BY id DESC
        LIMIT 1
        """
    )
    if not rows:
        raise RuntimeError("No se pudo recuperar la orden recién creada.")

    order = rows[0]
    create_production_orders(order["id"], validated["productos"])
    return order

In [78]:
def create_sale_order_from_payload(raw_payload: Dict[str, Any]):
    """
    Valida el payload bruto y crea una orden de venta.
    """
    validated = validate_sale_order_payload(raw_payload)
    return insert_sale_order(validated)

In [79]:
payload_test = {
  "id_vendedor": 1,
  "id_cliente": 1,
  "productos": [
    {"id_producto": 1, "cantidad": 3},
    {"id_producto": 2, "cantidad": 1}
  ],
  "fecha_entrega_solicitada": "2025-10-01"
}

In [80]:
try:
    conn = get_connection()
    resultado = create_sale_order_from_payload(payload_test)
finally:
    conn.close()

In [81]:
resultado

{'id': 5,
 'valor_total_pedido': Decimal('59000.00'),
 'estado': 'pendiente',
 'fecha_pedido': datetime.datetime(2025, 9, 28, 3, 20, 51, 180681, tzinfo=datetime.timezone.utc)}

# POST estado Solicitud venta

In [87]:
VALID_STATES = {
    "pendiente",
    "confirmada",
    "cancelada",
    "en_produccion",
    "lista",
    "entregada",
}

class ValidationError(Exception):
    """
    Error de validación para payloads de orden de venta.
    """

def update_sale_order_status(payload: Dict[str, Any]) -> Dict[str, Any]:
    required = ["id_pedido", "estado"]
    missing = [key for key in required if not payload.get(key)]
    if missing:
        raise ValidationError(f"Faltan campos obligatorios: {', '.join(missing)}")

    order_id = int(payload["id_pedido"])
    new_status = str(payload["estado"]).lower()

    if new_status not in VALID_STATES:
        raise ValidationError(
            f"Estado inválido: {new_status}. Valores permitidos: {', '.join(sorted(VALID_STATES))}"
        )

    rows = run_query(f"SELECT estado FROM {ENV}.orden_venta WHERE id = {order_id}")
    if not rows:
        raise ValidationError("La orden de venta indicada no existe.")

    current_status = rows[0]["estado"]
    if current_status == new_status:
        raise ValidationError(
            f"La orden {order_id} ya estaba en estado '{new_status}'."
        )

    run_command(
        f"UPDATE {ENV}.orden_venta SET estado = '{new_status}' WHERE id = {order_id}"
    )

    if new_status == "cancelada":
        run_command(
            f"UPDATE {ENV}.orden_produccion "
            f"SET estado = 'cancelada' "
            f"WHERE id_orden_venta = {order_id}"
        )
    elif new_status == "en_produccion":
        run_command(
            f"UPDATE {ENV}.orden_produccion "
            f"SET estado = 'en_proceso' "
            f"WHERE id_orden_venta = {order_id}"
        )
    elif new_status == "lista":
        run_command(
            f"""
            UPDATE {ENV}.orden_produccion
            SET estado = 'finalizada',
                fecha_fin = NOW()
            WHERE id_orden_venta = {order_id}
            """
        )

    return {
        "statusCode": 200,
        "body": f"Estado de la orden {order_id} actualizado de '{current_status}' a '{new_status}'.",
    }


In [92]:
input_json = {"id_pedido": 5, "estado": "lista"}

In [93]:
conn = get_connection()
try:
    response = update_sale_order_status(input_json)
except ValidationError as exc:
    response = {"statusCode": 400, "body": str(exc)}
finally:
    conn.close()

print(response)

{'statusCode': 400, 'body': "La orden 5 ya estaba en estado 'lista'."}


# POST estado orden de produccion/trabajo

In [ ]:
# hace falta? los estados se actualizan directamente en pedido y en consecuencia se actualizan solos en orden de produccion

: 

In [ ]:
PROD_STATES = {
    "planificada",
    "en_proceso",
    "cancelada",
}

def update_production_order_status(payload: Dict[str, Any]) -> Dict[str, Any]:
    """
    Actualiza el estado de una orden_produccion usando run_query/run_command.
    """
    required = ["id_orden", "estado"]
    missing = [key for key in required if not payload.get(key)]
    if missing:
        raise ValidationError(f"Faltan campos obligatorios: {', '.join(missing)}")

    order_id = int(payload["id_orden"])
    new_status = str(payload["estado"]).lower()

    if new_status not in PROD_STATES:
        raise ValidationError(
            f"Estado inválido: {new_status}. Valores permitidos: {', '.join(sorted(PROD_STATES))}"
        )

    rows = run_query(f"SELECT estado FROM {ENV}.orden_produccion WHERE id = {order_id}")
    if not rows:
        raise ValidationError("La orden de producción indicada no existe.")

    current_status = rows[0]["estado"]
    if current_status == new_status:
        raise ValidationError(f"La orden {order_id} ya estaba en estado '{new_status}'.")

    run_command(
        f"UPDATE {ENV}.orden_produccion SET estado = '{new_status}' WHERE id = {order_id}"
    )

    return {
        "statusCode": 200,
        "body": f"Estado de la orden de producción {order_id} actualizado de '{current_status}' a '{new_status}'.",
    }

In [ ]:
input_json = {"id_orden": 3, "estado": "en_proceso"}

In [60]:
conn = get_connection()
try:
    response = update_production_order_status(input_json)
except ValidationError as exc:
    response = {"statusCode": 400, "body": str(exc)}
finally:
    conn.close()

print(response)

{'statusCode': 400, 'body': 'La orden de producción indicada no existe.'}
